<a href="https://colab.research.google.com/github/gverafei/artificial-networks-technologies/blob/main/tarea6/tarea6_elu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tarea 6 (ELU)**

**Descripción**
Considerando el proyecto de colab proporcionado en la tarea:

+ Cambia `k = 10` y observa cómo varía la media/DesvStd.
+ Modifica `hidden_units=(128,64)` y/o `dropout=0.2`.
+ Cambia la función de activación: `relu` → `elu` o `gelu` (requiere `keras.layers.Activation('gelu')` en TF recientes).
+ Evalúa *F1-score* con `from sklearn.metrics import f1_score` 



## A. Configure virtual environment

Estas línea solo se ejecutan la primera vez. Solicitará crear un virtual environment. En un notebook en línea como Colab, pedirá selecciona el kernel de la esquina superior derecha.

In [1]:
!python3 -m venv .venv

In [2]:
!source .venv/bin/activate # Linux/Mac
# !.\venv\Scripts\activate # Windows

In [3]:
!pip install --upgrade pip --quiet

## B. Instala dependencias necesarias

In [4]:
!pip install numpy --quiet

In [5]:
!pip install matplotlib --quiet

In [6]:
!pip install tensorflow --quiet

In [7]:
!pip install scikit-learn --quiet

In [8]:
!pip install pandas --quiet

In [9]:
!pip install --upgrade certifi --quiet

## C. Configura el ambiente de pandas

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 50)
pd.set_option('display.precision', 3)
pd.set_option('display.width', 800)

print("Versiones -> pandas:", pd.__version__)

Versiones -> pandas: 2.3.3



# Validación cruzada vs Holdout (TensorFlow + utilidades de scikit-learn)

**Objetivo:** comparar *holdout* y *validación cruzada* evaluando un perceptrón multicapa (MLP) construido con **TensorFlow/Keras**, utilizando **scikit-learn** sólo para utilidades de particionado, *scaling* y métricas.

**Puntos clave (sin fuga de datos):**
- El *scaler* (p. ej., `StandardScaler`) se **ajusta únicamente con los datos de entrenamiento** en cada split.
- En K-Fold, **se re‑entrena el modelo desde cero por fold**.
- Se reporta la media y desviación estándar del desempeño.

> Dataset: **Breast Cancer Wisconsin** (clasificación binaria) de `sklearn.datasets`.



## 1. Setup e imports


In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score # Se agrega f1_score que pide la tarea
from sklearn.utils import shuffle

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

print(tf.__version__)


2.20.0



## 2. Carga de datos


In [12]:

data = load_breast_cancer()
X, y = data.data.astype(np.float32), data.target.astype(np.int32)
feature_names = data.feature_names
n_features = X.shape[1]
n_classes = len(np.unique(y))

X, y = shuffle(X, y, random_state=SEED)
X.shape, y.shape, n_features, n_classes


((569, 30), (569,), 30, 2)

In [13]:
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y

print("=== Muestras del dataset Breast Cancer ===")
print(df.head(5))

print("\nNúmero de características:", n_features)
print("Clases:", data.target_names.tolist())

=== Muestras del dataset Breast Cancer ===
   mean radius  mean texture  mean perimeter  mean area  mean smoothness  mean compactness  mean concavity  mean concave points  mean symmetry  mean fractal dimension  radius error  texture error  perimeter error  area error  smoothness error  compactness error  concavity error  concave points error  symmetry error  fractal dimension error  worst radius  worst texture  worst perimeter  worst area  worst smoothness  worst compactness  worst concavity  worst concave points  worst symmetry  worst fractal dimension  target
0        12.47         18.60           81.09      481.9            0.100             0.106           0.080                0.038          0.192                   0.064         0.396          1.044            2.497       30.29             0.007              0.019            0.027                 0.010           0.018                    0.004         14.97          24.64            96.05       677.9             0.143              0


## 3. Función para construir el modelo en TensorFlow/Keras

- Arquitectura MLP sencilla.
- `input_shape = (n_features,)`
- `EarlyStopping` para evitar sobreajuste.


In [14]:

def build_model(input_dim: int, hidden_units=(64, 32), dropout=0.0, learning_rate=1e-3):
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(hidden_units[0], activation='elu'),   ## Cambio que pide la tarea a "elu" y "gelu"
        layers.Dropout(dropout) if dropout > 0 else layers.Layer(),
        layers.Dense(hidden_units[1], activation='elu'),
        layers.Dense(1, activation='sigmoid')  # binaria
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model



## 4. Holdout correcto (train/test split) sin fuga de datos

- Dividir datos (80--20).
- Ajustar `StandardScaler` **solo con train**.
- Entrenar y evaluar en test.


Estandarización con StandarScaler
\begin{equation}
X' = \frac{X - \mu}{\sigma}
\end{equation}

Cuando aplicamos fit, aprende los parámetros usados los datos de prueba. Y luego los usa para la evaluación:

\begin{equation}
X_{scaled} = \frac{X - \mu_{train}}{\sigma_{train}}
\end{equation}

In [15]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

# Scaling sin fuga de datos
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

model = build_model(input_dim=X_train_s.shape[1], hidden_units=(64,32), dropout=0.1, learning_rate=1e-3)

#podemos cambiar val_loss <-- val_accuracy
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

history = model.fit(
    X_train_s, y_train,
    validation_split=0.2,  # validación interna para early stopping (dentro del train)
    epochs=200, batch_size=32, verbose=0, callbacks=[early_stopping]
)

# Evaluación en test
y_pred_proba = model.predict(X_test_s).ravel()
y_pred = (y_pred_proba >= 0.5).astype(int)

acc_holdout = accuracy_score(y_test, y_pred)
print(f"Accuracy (holdout, test): {acc_holdout:.4f}\n")

# Cálculo del F1 Score que pide la tarea
f1 = f1_score(y_test, y_pred)
print(f"F1 Score (holdout, test): {f1:.4f}\n")

print("Reporte de clasificación (holdout):")
print(classification_report(y_test, y_pred, target_names=data.target_names))


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Accuracy (holdout, test): 0.9211

F1 Score (holdout, test): 0.9379

Reporte de clasificación (holdout):
              precision    recall  f1-score   support

   malignant       0.90      0.88      0.89        42
      benign       0.93      0.94      0.94        72

    accuracy                           0.92       114
   macro avg       0.92      0.91      0.91       114
weighted avg       0.92      0.92      0.92       114




## 5. Validación Cruzada (Stratified K-Fold)

- Se crea un `StratifiedKFold` con `k=5` (puedes cambiarlo a 10).
- En **cada fold**:
  1. Se ajusta un scaler **con los datos de entrenamiento del fold**.
  2. Se construye y entrena **un nuevo modelo** desde cero.
  3. Se evalúa en el *validation fold*.
- Se reporta media y desviación estándar.


In [16]:

k = 10   # Cambio aquí para k-fold CV
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=SEED)

fold_accs = []
fold_f1_scores = []  # Lista para almacenar F1 Scores por fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), start=1):
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]

    # Scaling sin fuga: fit en train del fold, transform en train/val
    scaler = StandardScaler()
    X_tr_s = scaler.fit_transform(X_tr)
    X_val_s = scaler.transform(X_val)

    model = build_model(input_dim=X_tr_s.shape[1], hidden_units=(128,64), dropout=0.2, learning_rate=1e-3)
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(
        X_tr_s, y_tr,
        validation_data=(X_val_s, y_val),
        epochs=200, batch_size=32, verbose=0, callbacks=[early_stopping]
    )

    y_val_pred = (model.predict(X_val_s).ravel() >= 0.5).astype(int)
    acc = accuracy_score(y_val, y_val_pred)
    fold_accs.append(acc)
    f1 = f1_score(y_val, y_val_pred)  # Cálculo del F1 Score por fold
    fold_f1_scores.append(f1) # Almacenar F1 Score por fold
    print(f"Fold {fold}: accuracy = {acc:.4f}, F1 Score = {f1:.4f}")

print("\nResultados CV (k=%d):" % k)
print("Accuracies por fold:", [f"{a:.4f}" for a in fold_accs])
print("F1 Scores por fold:", [f"{f:.4f}" for f in fold_f1_scores])
print("Media = %.4f, DesvStd = %.4f" % (np.mean(fold_accs), np.std(fold_accs)))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Fold 1: accuracy = 1.0000, F1 Score = 1.0000
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/stepWARNING:tensorflow:6 out of the last 8 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x122e879c0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Fold 2: accuracy = 1.0000, F1 Score = 1.0000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Fold 3: accuracy = 1.0000, F1 Score = 1.


## 6. ¿Cuándo usar cada método?

- **Holdout**: rápido y suficiente cuando tienes *muchos datos* y sólo necesitas una estimación gruesa.
- **K-Fold**: preferible con *pocos/medianos datos* o cuando comparas modelos/hiperparámetros para tener una estimación más estable.



---

### Nota sobre fuga de datos (*data leakage*)
- **Escalado/normalización** SIEMPRE se ajusta sólo con *train*.
- En *cross‑validation*, esto se repite **dentro de cada fold**.
- Nunca uses información del conjunto de prueba/validación para preparar los datos de entrenamiento.
